## 1. Предобработка корпуса

* проводим лемматизацию и удаляем стоп-слова;
* приводем все леммы к нижнему регистру;
* добавляем чстеречные теги к словам.

Для предобработки мы будем использовать [*UDPipe*](https://ufal.mff.cuni.cz/udpipe)

In [1]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wget
import sys

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'

modelfile = wget.download(udpipe_url)
print('ok')

ok


Функция для предобработки текста

In [3]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    continue
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

print('ok')


ok


In [6]:
pip install ufal.udpipe

  Obtaining dependency information for ufal.udpipe from https://files.pythonhosted.org/packages/89/f7/4f041bd08ddb6b1198c0de71fa8e03610a129d47794ca360ea579b17296c/ufal.udpipe-1.3.1.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/892.6 kB ? eta -:--:--
   ---------------------------------------- 10.2/892.6 kB ? eta -:--:--
   - ------------------------------------- 30.7/892.6 kB 330.3 kB/s eta 0:00:03
   -- ------------------------------------ 61.4/892.6 kB 469.7 kB/s eta 0:00:02
   ------ ------------------------------- 153.6/892.6 kB 838.4 kB/s eta 0:00:01
   ------------ --------------------------- 286.7/892.6 kB 1.3 MB/s eta 0:00:01
   ------------------ --------------------- 409.6/892.6 kB 1.5 MB/s eta 0:00:01
   ------------------- -------------------- 430.1/892.6 kB 1.3 MB/s eta 0:00:01
   ----------------------------------- ---- 798.7/892.6 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------  890.9/892.6 kB 2.2 MB/s eta 0:00:01

In [4]:
from ufal.udpipe import Model, Pipeline
import os
import re

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    cnt = 0
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    lines = text.split('\n')
    tagged = []
    for line in lines:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        tagged_line = ' '.join(output)
        tagged.append(tagged_line)
        cnt += 1
        if cnt%1000 == 0:
            print(cnt)
    return '\n'.join(tagged)

Работа с корпусом

In [5]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Одна строка - одно предложение

In [6]:
f = 'my_text.txt'
data = gensim.models.word2vec.LineSentence(f)

2. Приступаем к обучению моделей

2.1 CBOW

Инициализируем модель. Параметры в скобочках:
* data - данные,
* size - размер вектора,
* window - размер окна наблюдения,
* min_count - мин. частотность слова в корпусе, которое мы берем,
* sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

In [11]:
model_CBOW = gensim.models.Word2Vec(data, vector_size=500, window=5, min_count=2, sg=0, encoding='cp1251')

TypeError: Word2Vec.__init__() got an unexpected keyword argument 'encoding'

Сколько слов в модели?

In [ ]:
print(len(model_CBOW.wv.vocab))

13641


Сохраняем модель

In [ ]:
model_CBOW.save('cbow.model')

2021-12-19 17:21:01,159 : INFO : saving Word2Vec object under cbow.model, separately None
2021-12-19 17:21:01,162 : INFO : not storing attribute vectors_norm
2021-12-19 17:21:01,164 : INFO : not storing attribute cum_table
2021-12-19 17:21:01,803 : INFO : saved cbow.model


Загружаем сохраненную модель

In [ ]:
from gensim.models import Word2Vec
cbow = Word2Vec.load('cbow.model')

Косинусное сходство

In [ ]:
cbow.wv.similarity("заболевание_NOUN", "болезнь_NOUN")

0.93937075

In [ ]:
cbow.wv.similarity("сердце_NOUN", "металлический_ADJ")

0.5333425

In [ ]:
cbow.wv.similarity("помощь_NOUN", "врач_NOUN")

0.87312686

In [ ]:
cbow.wv.similarity("корень_NOUN", "болезнь_NOUN")

0.84338915

Евклидово расстояние

In [ ]:
import numpy as np

euclid1 = np.linalg.norm(cbow.wv['симптом_NOUN'] - cbow.wv['насморк_NOUN'])
euclid2 = np.linalg.norm(cbow.wv['симптом_NOUN'] - cbow.wv['дерево_NOUN'])
print(euclid1, euclid2)

0.47760105 0.8164908


## 2.2 Skip-Gram

In [ ]:
model_sg = gensim.models.Word2Vec(data, size=500, window=5, min_count=2, sg=1)

In [ ]:
model_sg.save('skip-gram.model')

2021-12-19 17:37:07,246 : INFO : saving Word2Vec object under skip-gram.model, separately None
2021-12-19 17:37:07,249 : INFO : not storing attribute vectors_norm
2021-12-19 17:37:07,251 : INFO : not storing attribute cum_table
2021-12-19 17:37:07,842 : INFO : saved skip-gram.model


In [ ]:
from gensim.models import Word2Vec
sg = Word2Vec.load('skip-gram.model')

In [ ]:
for t in sg.most_similar(positive=[u'болезнь_NOUN'], topn=10):
    print (t[0], t[1])

язвенный_ADJ 0.818590521812439
гастрит_NOUN 0.795746386051178
заболевание_NOUN 0.7902160882949829
хронический_ADJ 0.7725076675415039
варикозный_ADJ 0.7723581790924072
панкреатит_VERB 0.7706602811813354
панкреатит_NOUN 0.76788330078125
желчнокамять_ADJ 0.7672724723815918
крона_PROPN 0.7628512382507324
анамнез_ADP 0.7602263689041138


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
sg.wv.similarity("заболевание_NOUN", "болезнь_NOUN")

0.7902161

In [ ]:
sg.wv.similarity("сердце_NOUN", "металлический_ADJ")

0.4050644

In [ ]:
sg.wv.similarity("помощь_NOUN", "врач_NOUN")

0.6428397

In [ ]:
sg.wv.similarity("корень_NOUN", "болезнь_NOUN")

0.34779614

In [ ]:
euclid3 = np.linalg.norm(sg.wv['симптом_NOUN'] - sg.wv['насморк_NOUN'])
euclid4 = np.linalg.norm(sg.wv['симптом_NOUN'] - sg.wv['дерево_NOUN'])
print(euclid3, euclid4)

2.0199683 2.9246976


## Коллокаты

In [ ]:
import re

for t in sg.most_similar(positive=[u'болезнь_NOUN'], topn=10):
  cond = re.search(r'_(NOUN)|(ADJ)|(NUM)', t[0])
  if cond != None:
    print (t[0], t[1])

язвенный_ADJ 0.818590521812439
гастрит_NOUN 0.795746386051178
заболевание_NOUN 0.7902160882949829
хронический_ADJ 0.7725076675415039
варикозный_ADJ 0.7723581790924072
панкреатит_NOUN 0.76788330078125
желчнокамять_ADJ 0.7672724723815918


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
